## Solving bidomain equations on rectangular domain with modified FitzHugh-Nagumo model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-misica


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = rectangle(2, 2, 100, 100)
reduce_factor = 1

In [3]:
class Model(MonodomainModel):
    def initial_V_m(self):
        def value(x):
            mu, alpha = 0.2, 50
            return -85 + 35 / (1 + np.exp((x[0] - mu) * alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector([1, 0])

        # Healthy conductivity
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        x_c, width = 1.0, 0.5
        sharpness = 50.0

        def reduce(x):
            return 1 + (reduce_factor - 1) / (
                1 + ufl.cosh(sharpness * (x[0] - x_c)) / ufl.cosh(sharpness * width)
            ) / (1 + ufl.cosh(sharpness * (x[1] - x_c)) / ufl.cosh(sharpness * width))

        def value(x):
            return 1 + (reduce_factor - 1) / (
                1 + np.cosh(sharpness * (x[0] - x_c)) / np.cosh(sharpness * width)
            ) / (1 + np.cosh(sharpness * (x[1] - x_c)) / np.cosh(sharpness * width))

        self.M_i = self.M_i / reduce(self.x)
        fun = fem.Function(self.V1)
        fun.interpolate(value)
        return fun

In [4]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.initial_V_m(),
    camera_direction="xy",
    cmap='PiYG',
    save_to="rectangle_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:36643/index.html?ui=P_0x7f1869baad10_0&reconnect=auto" class="pyvi…

In [6]:
plot_function(
    model.ischemia(),
    camera_direction="xy",
    cmap="RdYlGn",
    save_to="rectangle_ischemia_location.pdf",
    remove_scalar_bar = True
)

Widget(value='<iframe src="http://localhost:36643/index.html?ui=P_0x7f18abeaf650_1&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [7]:
model.solve(
    T=1000,
    steps=1000,
    save_to=f"rectangle_MFN_{reduce_factor}_red.mp4",
    camera_direction="xy",
    checkpoints=[int(1000 * i / 10) for i in range(1, 11)],
    checkpoint_file=f"rectangle_MFN_{reduce_factor}_red",
)

Solving problem:   4%|▍         | 39/1000 [00:04<02:00,  7.99it/s]